In [0]:
WITH target_system AS (
    SELECT coords.x AS tx, coords.y AS ty, coords.z AS tz
    FROM edsm.silver.systemsWithCoordinates
    WHERE name = :`Commander System`
),
stations_with_item AS (
    SELECT
        s.id,
        s.name,
        s.systemId,
        s.distanceToArrival,
        o.name AS outfitting_item
    FROM edsm.silver.stations s
    LATERAL VIEW explode(s.outfitting) t AS o
    WHERE o.name ILIKE CONCAT('%', :`Item`, '%')
),
filtered_systems AS (
    SELECT *
    FROM edsm.silver.systemsWithCoordinates swc
    CROSS JOIN target_system t
    WHERE ABS(swc.coords.x - t.tx) <= 25
      AND ABS(swc.coords.y - t.ty) <= 25
      AND ABS(swc.coords.z - t.tz) <= 25
),
joined AS (
    SELECT
        fs.name AS system_name,
        s.name AS station_name,
        s.outfitting_item,
        s.distanceToArrival,
        fs.coords.x AS x,
        fs.coords.y AS y,
        fs.coords.z AS z,
        SQRT(
            POW(fs.coords.x - t.tx, 2) +
            POW(fs.coords.y - t.ty, 2) +
            POW(fs.coords.z - t.tz, 2)
        ) AS distance
    FROM stations_with_item s
    JOIN filtered_systems fs
      ON s.systemId = fs.id
    CROSS JOIN target_system t
),
limited AS (
    SELECT
        outfitting_item,
        system_name,
        station_name,
        distance,
        distanceToArrival
    FROM joined
    ORDER BY distance ASC
    LIMIT 20
)
SELECT *
FROM limited
ORDER BY distanceToArrival ASC